# IBM Code Model Asset Exchange Human Pose Estimator

https://github.com/IBM/MAX-Human-Pose-Estimator

notes/links/etc

- `Netron`: https://github.com/lutzroeder/Netron
- `TF.js Converter`: https://github.com/tensorflow/tfjs-converter
- `TF.js API`: https://js.tensorflow.org/api/latest 


<br>

### Table of Contents

1. [Prerequisites](#Prerequisites)
1. [Run inference using the repo](#Run-inference-using-the-repo)
1. [Print the graph nodes](#Print-the-graph-nodes)
1. [Convert the model to a web-friendly format](#Convert-the-model-to-a-web-friendly-format)


<br>

## Prerequisites

- Clone the MAX Human Pose Estimator GitHub repository:

    ```
    git clone https://github.com/IBM/MAX-Human-Pose-Estimator.git
    ```
    

- Download and extract the model artifacts for the Human Pose Estimator:

    [http://max-assets.s3-api.us-geo.objectstorage.softlayer.net/human-pose-estimator/1.0/assets.tar.gz](http://max-assets.s3-api.us-geo.objectstorage.softlayer.net/human-pose-estimator/1.0/assets.tar.gz)

    For example, from a terminal window:

    ```
    curl -O http://max-assets.s3-api.us-geo.objectstorage.softlayer.net/human-pose-estimator/1.0/assets.tar.gz

    tar -zxvf assets.tar.gz
    ```

- If you want to run the MAX Human Pose Estimator locally on your machine, you will need to build the `pafprocess`:

    1. Install `swig`. For Mac OS X, you can install `swig` via `homebrew`
    
    ```
    brew install swig
    ```
    
    1. From a terminal window, go to the `/pafprocess` in the repo:
    
    ```
    cd <human-pose-estimator-repo>/core/tf_pose/pafprocess
    ```
    
    1. Run the command:
    
    ```
    swig -python -c++ pafprocess.i && python3 setup.py build_ext --inplace
    ```

<br>


In [ ]:
# This notebook has been tested with Python version 3.6.6
!python --version


In [ ]:
# This notebook has been tested with tensorflow 1.12.0, tensorflowjs 0.8.0, and numpy 1.15.1, opencv-python 3.4.4.19
!pip show tensorflow tensorflowjs numpy opencv-python


In [ ]:
# Uncomment to install the packages needed

# !pip install -Iv tensorflow
# !pip install -Iv tensorflowjs
# !pip install -Iv numpy
# !pip install -Iv opencv-python

# Restart the kernel after installation completes.


<br>

<strong>NOTE</strong>: Update the variables with the appropriate directory path


In [ ]:
# full path to cloned repo
human_pose_estimator = '/Users/va/Desktop/max/repos/MAX-Human-Pose-Estimator'

# full path to extracted frozen graph
frozen_graph_path = '/Users/va/Desktop/max/human-pose-estimator/model/human-pose-estimator-tensorflow.pb'


<br>
<hr>

# Run inference using the repo


In [ ]:
import sys

# add the repo to the Python path
sys.path.append(human_pose_estimator)


<br>

Load the model

In [ ]:
import config
import core
from core.tf_pose.estimator import TfPoseEstimator

model = TfPoseEstimator(frozen_graph_path, target_size=config.DEFAULT_IMAGE_SIZE)


<br>

<strong>NOTE</strong>: Update the variables with the appropriate path to an image to test

In [ ]:
# full path to a test image
test_image_path = '/Users/va/Desktop/max/test/img-03.jpg'


<br>

Run prediction on the image

In [ ]:
from PIL import Image
import io
import numpy as np
import pprint

pp = pprint.PrettyPrinter(indent=2)

def read_image(image_data):
    image = Image.open(io.BytesIO(image_data))
    image = np.array(image)[:,:,::-1] # Convert RGB to BGR for OpenCV
    return image
    

In [ ]:
# run prediction on an image
with open(test_image_path, 'rb') as image:
    img_array = read_image(image.read())
    humans = model.inference(img_array, resize_to_default=True, upsample_size=4.0)
    results = TfPoseEstimator.draw_human_pose_connection(img_array, humans)
    pp.pprint(results)
    

<br>

Visualize the detected poses


In [ ]:
import cv2
from matplotlib import pyplot as plt

# Define the colors for different human body parts.
CocoColors = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], 
              [170, 255, 0], [85, 255, 0], [0, 255, 0], [0, 255, 85], 
              [0, 255, 170], [0, 255, 255], [0, 170, 255], [0, 85, 255], 
              [0, 0, 255], [85, 0, 255], [170, 0, 255], [255, 0, 255], 
              [255, 0, 170], [255, 0, 85]]


# Each person may have multiple pose lines. Each pose line contains four coordinates 
# for the start and end points as [x1, y1, x2, y2]. The `cv2.line(img, (x1, y1), 
# (x2, y2), color, thickness)` function is utilized to visualize the detected
# pose lines.
def draw_pose(humans, img):
    for human in humans:
        pose_lines = human['pose_lines']
        for i in range(len(pose_lines)):
            line = pose_lines[i]['line']
            cv2.line(img, (line[0], line[1]), (line[2], line[3]), CocoColors[i], 3)


In [ ]:
# visualize the detected poses overlaid on the original image.

org_img = cv2.imread(test_image_path)[:,:,::-1]

fig = plt.figure()
fig.set_size_inches(18.5, 10.5)

plt.subplot(1, 3, 1)
plt.imshow(org_img)
plt.title("The original image")

pose_img = np.zeros(org_img.shape, dtype=np.uint8)
draw_pose(results, pose_img)
plt.subplot(1, 3, 2)
plt.imshow(pose_img)
plt.title("The detected poses")

overlaid_img = org_img.copy()
draw_pose(results, overlaid_img)
plt.subplot(1, 3, 3)
plt.imshow(overlaid_img)
plt.title("Poses overlaid on original image")

plt.show()

<br>
<hr>

# Print the graph nodes


In [ ]:
import tensorflow as tf

print('TF versions:', tf.GIT_VERSION, tf.VERSION)


<br>

Load the graph

In [ ]:
# load the frozen file and parse it to get the unserialized graph_def
def load_frozen_graph(graph_path):
    with tf.gfile.GFile(graph_path, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        return graph_def


In [ ]:
# load graph
restored_graph_def = load_frozen_graph(frozen_graph_path)


<br>

Print graph nodes

In [ ]:
# print list graph nodes/tensors
def list_nodes(graph_def):
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(
            graph_def,
            input_map=None,
            return_elements=None,
            name=""
        )

    sess = tf.Session(graph=graph)
    nodes = sess.graph.as_graph_def().node
    print('graph has {} nodes \r\n'.format(len(nodes)))
    
    for n in nodes:
        print(n.name + '=>' +  n.op)


In [ ]:
# list graph nodes
list_nodes(restored_graph_def)


<br>
<hr>

# Convert the model to a web-friendly format

[https://github.com/tensorflow/tfjs-converter](https://github.com/tensorflow/tfjs-converter)


```
tensorflowjs_converter \
    --input_format=tf_frozen_model \
    --output_node_names='Openpose/concat_stage7' \
    /path/to/frozen/model.pb \
    /path/to/web_asset_output_dir
```


In [ ]:
# check for tensorflowjs_converter
!tensorflowjs_converter --version


<br>

<strong>NOTE</strong>: Update the variables with the appropriate graph output nodes and path to save the converted model assets

In [ ]:
# set the graph output nodes
graph_output_nodes = 'Openpose/concat_stage7'

# set appropriate desired output path for web format
web_asset_dir = '/Users/va/Desktop/max/human-pose-estimator/model-tfjs'


In [ ]:
import pathlib

# create directory if it does not exist
pathlib.Path(web_asset_dir).mkdir(parents=True, exist_ok=True)


<br>

Run the converter


In [ ]:

!tensorflowjs_converter \
    --input_format=tf_frozen_model \
    --output_node_names={graph_output_nodes} \
    {frozen_graph_path} \
    {web_asset_dir}


<br>

Print the converted model assets


In [ ]:
import os
import time

print("Web asset directory {}:".format(web_asset_dir))

web_assets = os.listdir(web_asset_dir)
web_assets.sort()

for file in web_assets:
    file_stat = os.stat("{}/{}".format(web_asset_dir,file))
    print(" {} {} {:>20}".format(file.ljust(30), time.ctime(file_stat.st_mtime), file_stat.st_size))
